---
format:
  html:
    code-fold: false
jupyter: python3
---

# Spatial feature engineering (part II) {#sec-spatial-feature-ii}

In this second part of Spatial Feature Engineering, we turn to Map Synthesis. There is only one [reading](https://geographicdata.science/book/notebooks/12_feature_engineering.html) to complete for this block, from the GDS Book

## Packages and modules

In [ ]:
import pandas, geopandas
import numpy as np
import contextily
import tobler
import matplotlib.pyplot as plt

## Data

If you want to read more about the data sources behind this dataset, head to the [Datasets](./data/datasets) section.

Assuming you have the file locally on the path `../data/`:

In [ ]:
pts = geopandas.read_file("../data/madrid_abb.gpkg")

We will be working with a modified version of `pts`:

-   Since we will require distance calculations, we will switch to the Spanish official projection
-   To make calculations in the illustration near-instantaneous, we will work with a smaller (random) sample of Airbnb properties (10% of the total)

In [ ]:
db = pts.sample(
    frac=0.1, random_state=123
).to_crs(epsg=25830)

As you can see in the description, the new CRS is expressed in metres:

In [ ]:
db.crs

## Distance buffers

*How many Airbnb's are within 500m of each Airbnb?*

In [ ]:
from pysal.lib import weights

Using `DistanceBand`, we can build a spatial weights matrix that assigns `1` to each observation within 500m, and `0` otherwise.

In [ ]:
w500m = weights.DistanceBand.from_dataframe(
    db, threshold=500, binary=True
)

The number of neighbors can be accessed through the `cardinalities` attribute:

In [ ]:
n_neis = pandas.Series(w500m.cardinalities)
n_neis.head()

In [ ]:
fig, ax = plt.subplots()

db.assign(
    n_neis=n_neis
).plot("n_neis", markersize=0.1, ax=ax)

plt.show()

::: callout-note
**Challenge:** Calculate the number of AirBnb properties within 250m of each other property. *What is the average?*
:::

## Distance rings

*How many Airbnb's are between 500m and 1km of each Airbnb?*

In [ ]:
w1km = weights.DistanceBand.from_dataframe(
    db, threshold=1000, binary=True
)

Now, we could do simply a subtraction:

In [ ]:
n_ring_neis = pandas.Series(w1km.cardinalities) - n_neis

Or, if we need to know *which is which*, we can use set operations on weights:

In [ ]:
w_ring = weights.w_difference(w1km, w500m, constrained=False)

And we can confirm they're both the same:

In [ ]:
(pandas.Series(w_ring.cardinalities) - n_ring_neis).sum()

::: callout-note
**Challenge:** Can you create a plot with the following two lines?

-   One depicting the average number of properties within a range of 50m, 100m, 250m, 500m, 750m
-   Another one with the *increase* of average neighbors for the same distances above
:::

## Cluster membership (points)

We can use the spatial configuration of observations to classify them as part of clusters or not, which can then be encoded, for example, as dummy variables in a model.

We will learn a method to identify clusters of points, based on their density across space. To do this, we will use the widely used `DBSCAN` algorithm. For this method, a cluster is a concentration of at least `min_pts` points, each of them within a distance `eps` of at least one other point in the cluster. Points in the dataset are then divided into three categories:

-   *Noise*, for those points outside a cluster.
-   *Cores*, for those points inside a cluster whith at least `min_pts` points in the cluster within distance `eps`.
-   *Borders* for points inside a cluster with less than `min_pts` other points in the cluster within distance `eps`.

Both `min_pts` and eps need to be prespecified by the user before running `DBSCAN`. This is a critical, as the value of these parameters can influence significantly the final result. For more on this, check [here](https://scikit-learn.org/stable/modules/clustering.html). Before exploring this in greater depth, let us get a first run at computing `DBSCAN` in Python, with `eps` = 500 and a minimum of `min_pct` = 2% of points for a candidate cluster to be considered so.

In [ ]:
from sklearn.cluster import DBSCAN

min_pct = 2
min_pts = len(db) * min_pct // 100
eps = 500

We will illustrate it with a minimum number of points of `min_pct` % of the sample and a maximum radious of `eps` metres.

In [ ]:
model = DBSCAN(min_samples=min_pts, eps=eps)

Once ready, we “fit” it to our data, but note that we first need to express the longitude and latitude of our points in metres.

In [ ]:
db['x'] = db.geometry.x
db['y'] = db.geometry.y
model.fit(
    db[['x', 'y']]
);

We will attach the labels to `db` for easy access:

In [ ]:
db["labels"] = model.labels_

We can define boundaries to turn point clusters into polygons if that fits our needs better:

The code in the next cell is a bit more advanced than expected for this course, but is used here as an illustration.

In [ ]:
from pysal.lib import cg

boundaries = []
cl_ids = [i for i in db["labels"].unique() if i!=-1]
for cl_id in cl_ids:
    sub = db.query(f"labels == {cl_id}")
    cluster_boundaries = cg.alpha_shape_auto(
        np.array(
            [sub.geometry.x, sub.geometry.y]
        ).T,
    )
    boundaries.append(cluster_boundaries)
boundaries = geopandas.GeoSeries(
    boundaries, index=cl_ids, crs=db.crs
)

And we can see what the clusters look like:

In [ ]:
fig, ax = plt.subplots()

db.to_crs(
    epsg=3857
).plot(
    markersize=0.1, color="lime", ax=ax
)
boundaries.to_crs(
    epsg=3857
).plot(
    ax=ax, edgecolor="red", facecolor="none"
)

contextily.add_basemap(
    ax,
    source=contextily.providers.CartoDB.DarkMatterNoLabels
)

plt.show()

::: callout-note
**Challenge:** *How does the map above change if you require 5% of points instead of 2% for a candidate cluster to be considered so?*
:::

## Cluster membership (polygons)

We can take a similar approach as above if we have polygon geographies instead of points. Rather than using DBSCAN, here we can rely on local indicators of spatial association (LISAs) to pick up spatial concentrations of high or low values.

For the illustration, we will aggregate the location of Airbnb properties to a regular hexagonal grid, similar to how we generated it when transferring from polygons to polygons. First we create a polygon covering the extent of points:

In [ ]:
one = geopandas.GeoSeries(
    [cg.alpha_shape_auto(
        np.array(
            [db.geometry.x, db.geometry.y]
        ).T,
    )],
    crs=db.crs
)

Then we can tessellate:

In [ ]:
abb_hex = tobler.util.h3fy(
    one, resolution=8
)

And obtain a count of points in each polygon:

In [ ]:
counts = geopandas.sjoin(
    db, abb_hex
).groupby(
    "index_right"
).size()

abb_hex["count"] = counts
abb_hex["count"] = abb_hex["count"].fillna(0)

fig, ax = plt.subplots()

abb_hex.plot("count", scheme="fisherjenks", ax=ax)

plt.show()

To identify spatial clusters, we rely on `esda`:

In [ ]:
from pysal.explore import esda

And compute the LISA statistics:

In [ ]:
w = weights.Queen.from_dataframe(abb_hex)
lisa = esda.Moran_Local(abb_hex["count"], w)

For a visual inspection of the clusters, `splot`:

In [ ]:
from pysal.viz import splot
from splot.esda import lisa_cluster

In [ ]:
lisa_cluster(lisa, abb_hex, p=0.01)
plt.show()

And, if we want to extract the labels for each polygon, we can do so from the `lisa` object:

In [ ]:
lisa.q * (lisa.p_sim < 0.01)

## Next steps

If you want a bit more background into some of the techniques reviewed in this block, the following might be of interest:

-   [This block](https://pietrostefani.github.io/gds/clustering.html) of the GDS Course [@courseGDS-pietrostefani-cabrera] will introduce you to more techniques like the LISAs seen above to explore the spatial dimension of the statistical properties of your data. If you want a more detailed read, [this Chapter](https://geographicdata.science/book/notebooks/04_spatial_weights.html) of the GDS Book [@reyABwolf] will do just that.
-   [This block](https://pietrostefani.github.io/gds/points.html) of the GDS Course [@courseGDS-pietrostefani-cabrera] will introduce you to more techniques for exploring point patterns. If you want a more comprehensive read, [this Chapter](https://geographicdata.science/book/notebooks/08_point_pattern_analysis.html) of the GDS Book [@reyABwolf] will do just that.